In [4]:
# ============================================
# COVID-19 백신 접종률과 치명률 상관관계 분석
# ============================================
import pandas as pd
from pathlib import Path
import plotly.graph_objects as go

# ============================================
# 1. 데이터 로드
# ============================================
data_dir = Path.cwd() / 'data'
PATH_VAC = data_dir / '예방접종_통계_현황_통합.csv'
PATH_LOC = data_dir / '시군구별_월별_확진자_사망_발생현황_통합.csv'

vac = pd.read_csv(PATH_VAC)
loc = pd.read_csv(PATH_LOC)

print("데이터 로드 완료")

# ============================================
# 2. 데이터 전처리
# ============================================

# 백신 접종률 데이터 처리
vac['날짜'] = pd.to_datetime(vac['날짜'])
vac['month'] = vac['날짜'].dt.to_period('M').astype(str)
vac_rate = vac[vac['지표'] == '접종률'].copy()
vac_m = vac_rate.groupby('month')['값'].mean().reset_index()
vac_m = vac_m.rename(columns={'값': 'vacc_rate'})

# 확진자 및 사망자 데이터 처리
loc['날짜'] = pd.to_datetime(loc['날짜'])
loc['month'] = loc['날짜'].dt.to_period('M').astype(str)

cases = loc[loc['유형'] == '확진자'].copy()
deaths = loc[loc['유형'] == '사망자'].copy()

cases_m = cases.groupby('month')['값'].sum().reset_index()
cases_m = cases_m.rename(columns={'값': 'total_cases'})

deaths_m = deaths.groupby('month')['값'].sum().reset_index()
deaths_m = deaths_m.rename(columns={'값': 'total_deaths'})

# 치명률 계산 (사망자/확진자 * 100)
severity = pd.merge(cases_m, deaths_m, on='month')
severity['cfr'] = (severity['total_deaths'] / severity['total_cases'] * 100)

# 최종 데이터 병합
final_df = pd.merge(vac_m, severity, on='month')
final_df = final_df.sort_values('month').reset_index(drop=True)

print(f"분석 기간: {final_df['month'].min()} ~ {final_df['month'].max()}")
print(f"총 {len(final_df)}개월 데이터")

# ============================================
# 3. 시각화 - 삼중 축 그래프
# ============================================

fig = go.Figure()

# 그래프 1: 백신 접종률 (파란색 선, 왼쪽 Y축)
fig.add_trace(go.Scatter(
    x=final_df['month'],
    y=final_df['vacc_rate'],
    name='백신 접종률',
    mode='lines+markers',
    line={'color': '#1E88E5', 'width': 3},
    marker={'size': 8, 'symbol': 'circle'},
    yaxis='y1',
    hovertemplate='%{x}<br>접종률: %{y:.1f}%<extra></extra>'
))

# 그래프 2: 월별 사망자 수 (빨간색 막대, 오른쪽 첫 번째 Y축)
fig.add_trace(go.Bar(
    x=final_df['month'],
    y=final_df['total_deaths'],
    name='월별 사망자',
    marker={'color': '#E53935', 'opacity': 0.5},
    yaxis='y2',
    hovertemplate='%{x}<br>사망자: %{y:,}명<extra></extra>'
))

# 그래프 3: 치명률 (주황색 점선, 오른쪽 두 번째 Y축)
fig.add_trace(go.Scatter(
    x=final_df['month'],
    y=final_df['cfr'],
    name='치명률',
    mode='lines+markers',
    line={'color': '#FB8C00', 'width': 3, 'dash': 'dot'},
    marker={'size': 8, 'symbol': 'diamond'},
    yaxis='y3',
    hovertemplate='%{x}<br>치명률: %{y:.2f}%<extra></extra>'
))

# 상관계수 계산
correlation = final_df['vacc_rate'].corr(final_df['cfr'])

fig.update_layout(
    # 제목
    title={
        'text': 'COVID-19 백신 접종률과 치명률 상관관계 분석<br>' +
                '<sub>월별 백신 접종률, 사망자 수, 치명률 추이</sub>',
        'font': {'size': 20, 'color': '#2C3E50', 'family': 'Arial'},
        'x': 0.5,
        'xanchor': 'center',
        'y': 0.96,
        'yanchor': 'top'
    },
    
    # X축 설정
    xaxis={
        'title': {'text': '시간 (월별)', 'font': {'size': 12}},
        'tickangle': -45,
        'showgrid': True,
        'gridcolor': 'rgba(200,200,200,0.2)',
        'tickfont': {'size': 9},
        'domain': [0, 0.87]
    },
    
    # Y축 1: 백신 접종률 (왼쪽)
    yaxis={
        'title': {'text': '백신 접종률 (%)', 'font': {'size': 11, 'color': '#1E88E5'}},
        'tickfont': {'color': '#1E88E5', 'size': 9},
        'side': 'left',
        'showgrid': True,
        'gridcolor': 'rgba(30, 136, 229, 0.1)',
        'range': [0, final_df['vacc_rate'].max() * 1.15]
    },
    
    # Y축 2: 사망자 수 (오른쪽 첫 번째)
    yaxis2={
        'title': {'text': '월별 사망자 수 (명)', 'font': {'size': 11, 'color': '#E53935'}},
        'tickfont': {'color': '#E53935', 'size': 9},
        'overlaying': 'y',
        'side': 'right',
        'anchor': 'x',
        'showgrid': False,
        'range': [0, final_df['total_deaths'].max() * 1.2]
    },
    
    # Y축 3: 치명률 (오른쪽 두 번째)
    yaxis3={
        'title': {'text': '치명률 (%)', 'font': {'size': 11, 'color': '#FB8C00'}},
        'tickfont': {'color': '#FB8C00', 'size': 9},
        'overlaying': 'y',
        'side': 'right',
        'anchor': 'free',
        'position': 0.97,
        'showgrid': False,
        'range': [0, final_df['cfr'].max() * 1.3]
    },
    
    # 기타 설정
    template='plotly_white',
    plot_bgcolor='white',
    paper_bgcolor='white',
    height=600,
    width=1400,
    hovermode='x unified',
    
    # 범례 설정
    legend={
        'orientation': 'h',
        'yanchor': 'top',
        'y': -0.18,
        'xanchor': 'center',
        'x': 0.435,
        'font': {'size': 11, 'family': 'Arial'},
        'bgcolor': 'rgba(255,255,255,0.9)',
        'bordercolor': '#CCCCCC',
        'borderwidth': 1
    },
    
    # 여백 설정
    margin={'l': 70, 'r': 130, 't': 110, 'b': 110}
)

# 5. 분석 결과 주석 추가

result_text = (
    f'<b>분석 결과</b><br>'
    f'분석 기간: {final_df["month"].min()} ~ {final_df["month"].max()}<br>'
    f'총 {len(final_df)}개월 데이터<br><br>'
    f'최고 접종률: {final_df["vacc_rate"].max():.1f}%<br>'
    f'총 사망자 수: {final_df["total_deaths"].sum():,.0f}명<br>'
    f'평균 치명률: {final_df["cfr"].mean():.2f}%<br><br>'
    f'<b>상관계수: {correlation:.4f}</b><br>'
    f'(음의 상관관계 확인)'
)

fig.add_annotation(
    text=result_text,
    xref="paper", yref="paper",
    x=0.01, y=0.92,
    showarrow=False,
    font={'size': 10, 'color': '#2C3E50', 'family': 'Arial'},
    bgcolor='rgba(248, 249, 250, 0.95)',
    bordercolor='#DEE2E6',
    borderwidth=2,
    borderpad=10,
    xanchor='left',
    yanchor='top',
    align='left'
)

interpretation_text = (
    f'<b>결과 해석</b><br>'
    f'백신 접종률이 증가함에 따라<br>'
    f'치명률이 감소하는 경향을 보임<br><br>'
    f'이는 백신이 중증 및 사망<br>'
    f'예방에 효과적임을 시사함'
)

fig.add_annotation(
    text=interpretation_text,
    xref="paper", yref="paper",
    x=0.01, y=0.45,
    showarrow=False,
    font={'size': 10, 'color': '#2C3E50', 'family': 'Arial'},
    bgcolor='rgba(232, 245, 233, 0.9)',
    bordercolor='#81C784',
    borderwidth=2,
    borderpad=10,
    xanchor='left',
    yanchor='top',
    align='left'
)

print("\n그래프 생성 완료")

# 7. 파일 저장

output_html = Path.cwd() / 'covid_analysis.html'
output_png = Path.cwd() / 'covid_analysis.png'

fig.write_html(str(output_html))
fig.write_image(str(output_png), width=1400, height=600, scale=2)

print(f"\n저장 완료:")
print(f"- HTML 파일: {output_html}")
print(f"- PNG 이미지: {output_png}")

# 8. 기초 통계 출력

print("\n" + "="*60)
print("기초 통계")
print("="*60)
print(f"분석 기간: {final_df['month'].min()} ~ {final_df['month'].max()}")
print(f"데이터 개수: {len(final_df)}개월")
print(f"\n백신 접종률:")
print(f"  - 최저: {final_df['vacc_rate'].min():.2f}%")
print(f"  - 최고: {final_df['vacc_rate'].max():.2f}%")
print(f"  - 평균: {final_df['vacc_rate'].mean():.2f}%")
print(f"\n사망자 수:")
print(f"  - 월 최소: {final_df['total_deaths'].min():,.0f}명")
print(f"  - 월 최대: {final_df['total_deaths'].max():,.0f}명")
print(f"  - 총 사망: {final_df['total_deaths'].sum():,.0f}명")
print(f"\n치명률:")
print(f"  - 최저: {final_df['cfr'].min():.2f}%")
print(f"  - 최고: {final_df['cfr'].max():.2f}%")
print(f"  - 평균: {final_df['cfr'].mean():.2f}%")
print(f"\n상관 분석:")
print(f"  - 접종률 vs 치명률 상관계수: {correlation:.4f}")
print(f"  - 해석: {'강한' if abs(correlation) > 0.7 else '중간' if abs(correlation) > 0.4 else '약한'} 음의 상관관계")
print("="*60)

데이터 로드 완료
분석 기간: 2021-02 ~ 2023-08
총 31개월 데이터

그래프 생성 완료

저장 완료:
- HTML 파일: /Users/gimoonee/Desktop/covid/covid_analysis.html
- PNG 이미지: /Users/gimoonee/Desktop/covid/covid_analysis.png

기초 통계
분석 기간: 2021-02 ~ 2023-08
데이터 개수: 31개월

백신 접종률:
  - 최저: 0.01%
  - 최고: 43.96%
  - 평균: 33.20%

사망자 수:
  - 월 최소: 58명
  - 월 최대: 8,171명
  - 총 사망: 34,501명

치명률:
  - 최저: 0.03%
  - 최고: 1.62%
  - 평균: 0.32%

상관 분석:
  - 접종률 vs 치명률 상관계수: -0.6922
  - 해석: 중간 음의 상관관계
